<a href="https://colab.research.google.com/github/leonammeta8154/Estela_IA/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService # memoria interna desse agente.
from google.adk.tools import google_search # ferramenta do google
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name ="agente_buscador",
      model="gemini-2.0-flash",
      description="agente de buscar informações no google",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do (google_search) para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
      Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
      Se um tema tiver poucas notícias ou reações entusiasmadas, é posspivel que ele não seja tão relevante aasim
      esse lançamenbto relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
      """
  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  # Executa o agente
  lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos_buscados

In [9]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""

        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lanlamentos mais recentes e revelantes buscador, você deve:
        usar a ferramenta de busca do google (google_search) para criar um plano sobre quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano com os asuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [10]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [11]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [12]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: academia
Maravilha! Vamos então criar o post sobre novidades em academia

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais recentes e relevantes no mundo da academia, focarei em notícias e novidades dos últimos 30 dias que geraram entusiasmo e cobertura significativa. Aqui estão as perguntas que usarei para guiar minha pesquisa:
> 
> 1.  Quais são os lançamentos de equipamentos de ginástica mais recentes e comentados?
> 2.  Quais são os novos aplicativos ou softwares de fitness que estão ganhando popularidade?
> 3.  Existem novas modalidades de treino ou programas de fitness que estão em alta?
> 4.  Quais são as últimas tendências em nutrição esportiva e suplementos?
> 
> 
> Com base nas minhas buscas, aqui estão alguns dos lançamentos e tendências mais relevantes no mundo da academia para 2025:
> 
> **Equipamentos e Tecnologias:**
> 
> *   **Equipamentos fitness com foco em sustentabilidade:** Há uma crescente demanda por equipamentos feitos com materiais recicláveis e processos de produção que reduzem o impacto ambiental.
> *   **Integração de tecnologia:** Academias estão investindo em soluções como telas interativas que mostram o desempenho em tempo real e equipamentos que se ajustam automaticamente.
> *   **Tecnologia vestível:** Dispositivos como smartwatches e sensores de performance fornecem informações valiosas para treinos personalizados, como zonas de frequência cardíaca e níveis de recuperação.
> 
> **Aplicativos e Softwares:**
> 
> *   **Aplicativos de exercícios:** Estão em alta, oferecendo treinos flexíveis, adaptáveis e personalizáveis. Em 2023, mais de 370 milhões de usuários utilizaram aplicativos fitness, e esse número tende a crescer.
> *   **GymRats:** Um novo aplicativo que promove a criação de desafios, a socialização e o senso de comunidade entre os praticantes de atividades físicas.
> *   **Google Fit:** Um aplicativo gratuito com atualizações frequentes e recursos excelentes para usuários de smartwatches Wear OS.
> 
> **Modalidades de Treino e Programas de Fitness:**
> 
> *   **Treinamento funcional personalizado:** Adaptação dos exercícios às necessidades individuais de cada pessoa, levando em conta fatores como idade, nível de condicionamento e objetivos específicos.
> *   **HIIT (Treino Intervalado de Alta Intensidade):** Continua popular, com novidades como a incorporação de elementos de mobilidade e recuperação ativa entre os intervalos de alta intensidade.
> *   **Treinamento em grupos pequenos:** Proporciona acompanhamento personalizado e incentivo coletivo, criando um senso de comunidade.
> *   **Treinos híbridos e funcionais:** Combinação de treinos de força com trabalho cardiovascular.
> *    **Programas de condicionamento físico para idosos:** Serviços fundamentais para a manutenção da saúde, prevenção de lesões e doenças crônicas, além de garantir uma rotina independente.
> *   **Treinamento de força tradicional:** Envolve o uso de equipamentos como barras, halteres e kettlebells.
> 
> **Nutrição Esportiva e Suplementos:**
> 
> *   **Nutrição personalizada:** Planos alimentares personalizados com base nas necessidades alimentares específicas, níveis de atividade e objetivos de saúde de cada indivíduo.
> *   **Nutrição esportiva baseada no veganismo:** Proteínas em pó, barras energéticas e suplementos de recuperação à base de plantas estão ganhando força.
> *   **Alimentação plant-based com mais funcionalidade:** Foco em alimentos funcionais que contribuam para melhorar a imunidade, a digestão e a saúde mental.
> *   **Suplementação personalizada:** Suplementos customizados com base no perfil genético e metabólico de cada indivíduo.
> *   **O papel do intestino na performance esportiva:** Dietas que equilibrem a microbiota intestinal, melhorando a digestão e a absorção de nutrientes.
> 
> Espero que esta informação seja útil!
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com certeza! Com base nos lançamentos e tendências que você pesquisou, podemos criar um plano de conteúdo interessante e relevante para um post sobre o mundo da academia em 2025.
> 
> **Tema Mais Relevante:** Integração de tecnologia, treinos personalizados e sustentabilidade no mundo fitness.
> 
> **Pontos Relevantes:**
> 
> *   **Equipamentos Sustentáveis:** Materiais recicláveis, menor impacto ambiental.
> *   **Tecnologia Integrada:** Telas interativas, ajustes automáticos.
> *   **Wearables:** Smartwatches e sensores para dados precisos.
> *   **Aplicativos:** Personalização, desafios, comunidade (GymRats, Google Fit).
> *   **Treino Personalizado:** Adaptação individual (idade, nível, objetivos).
> *   **HIIT:** Novidades com mobilidade e recuperação.
> *   **Pequenos Grupos:** Acompanhamento e incentivo.
> *   **Treinos Híbridos:** Força e cardio combinados.
> *   **Idosos:** Saúde, prevenção, independência.
> *   **Força Tradicional:** Barras, halteres, kettlebells.
> *   **Nutrição Personalizada:** Planos específicos.
> *   **Veganismo:** Proteínas e suplementos vegetais.
> *   **Plant-Based Funcional:** Imunidade, digestão, saúde mental.
> *   **Suplementação Personalizada:** Genética e metabolismo.
> *   **Intestino:** Microbiota e performance.
> 
> **Plano de Conteúdo para o Post:**
> 
> 1.  **Título:** "Revolução Fitness 2025: Tecnologia, Personalização e Sustentabilidade na Sua Academia"
> 2.  **Introdução:**
>     *   Visão geral das tendências fitness em 2025.
>     *   Ênfase na tecnologia, personalização e sustentabilidade como pilares.
> 3.  **Equipamentos e Tecnologia:**
>     *   Equipamentos sustentáveis e seus benefícios.
>     *   Integração de telas interativas e ajustes automáticos.
>     *   Como wearables otimizam o treino (smartwatches, sensores).
> 4.  **Aplicativos e Softwares:**
>     *   Crescimento dos aplicativos fitness (estatísticas).
>     *   Destaque para GymRats (desafios, comunidade) e Google Fit (recursos).
> 5.  **Modalidades de Treino:**
>     *   Treinamento funcional personalizado: importância e exemplos.
>     *   Novidades no HIIT (mobilidade e recuperação).
>     *   Benefícios dos treinos em pequenos grupos.
>     *   Treinos híbridos e funcionais para resultados completos.
>     *   Programas específicos para idosos (saúde e independência).
>     *   A importância do treinamento de força tradicional.
> 6.  **Nutrição Esportiva:**
>     *   Nutrição personalizada: como montar um plano.
>     *   Veganismo e alimentos plant-based na nutrição esportiva.
>     *   Suplementação personalizada com base em genética e metabolismo.
>     *   O papel da saúde intestinal na performance esportiva.
> 7.  **Conclusão:**
>     *   Recapitulação das tendências.
>     *   Encorajamento para experimentar as novidades.
>     *   Convite para interação nos comentários.
> 
> Com este plano, você poderá criar um post informativo, interessante e relevante sobre o mundo da academia em 2025, abordando as tendências mais quentes e como elas podem beneficiar os praticantes de atividades físicas.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🏋️‍♀️ O futuro fitness chegou e ele é TECNOLÓGICO, PERSONALIZADO e SUSTENTÁVEL! 🚀
> 
> Prepare-se para a revolução fitness 2025! 🤯 As academias estão se transformando com equipamentos eco-friendly, cheios de tecnologia e treinos feitos sob medida pra VOCÊ!
> 
> 🔋 **Equipamentos do futuro:** Já pensou em treinar em aparelhos feitos com materiais reciclados e com telas que te guiam em tempo real? A pegada sustentável chegou com tudo!
> 
> 📱 **Apps bombando:** Esqueça as planilhas! Com o GymRats e Google Fit, você tem desafios, comunidades e dados precisos na palma da mão. 📈
> 
> 💪 **Treino do seu jeito:** Chega de treinar igual todo mundo! A personalização é a chave: idade, nível, objetivos… tudo pensado pra você! E não para por aí: HIIT com foco em mobilidade, treinos em grupo e até programas especiais para nossos #AluraLovers +60! ✨
> 
> 🌱 **Nutrição turbinada:** Vegano? Plant-based? Intestino feliz? A nutrição esportiva está cada vez mais personalizada e focada em performance e bem-estar! 🥗
> 
> E aí, #AluraFit, preparado(a) para essa nova era? 😎 Conta pra gente nos comentários qual dessas tendências te anima mais! 👇
> 
> #Fitness2025 #TecnologiaNoEsporte #SustentabilidadeFitness
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


--------------------------------------------------------------
